In [1]:
import pandas as pd
from sklearn.cluster import KMeans
import seaborn as sns
from matplotlib import pyplot as plt 
import math
from ipynb.fs.full.Utils import createKDeplot, loadDataFrames, dropFeaturesWithNoVariance,loadCoverGroups
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy import stats
import dask_ml.cluster
import pandas as pd
import gower
from ipynb.fs.full.Pipeline import removeConstantFeatures, applyBoxCoxAllFeatures, centerAndScale, applyEuclideanDistance, getIndexAndVector, applyBoxCoxFeatures,applyJaccard, getEuclideanJaccardDf , applyCosineDistance, applyGower

In [34]:
df = pd.read_csv('./data/unified/05-allDataContinousCategoricalTransition.csv')
ids = df.id
df = df.iloc[:,2:]

In [35]:
df

,id,danceability,energy,speechiness,acousticness,liveness,valence,tempo,0,1,...,mood_5,mood_6,mood_7,mood_8,mood_9,mood_10,mood_11,instrumentalness_cat_0,instrumentalness_cat_1,instrumentalness_cat_2
0,23,0.494,0.756,0.0432,0.001400,0.1640,0.589,133.337,0.000000,0.000000,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,105,0.582,0.744,0.0336,0.000458,0.0405,0.674,133.344,0.185185,0.000000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,118,0.614,0.755,0.0276,0.016200,0.0831,0.645,92.622,0.000000,0.000000,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120,0.319,0.807,0.0388,0.001040,0.1310,0.601,118.097,0.029412,0.000000,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,292,0.496,0.527,0.0339,0.195000,0.1380,0.283,155.932,0.013889,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5972,6002496,0.509,0.537,0.0384,0.347000,0.0805,0.311,170.030,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5973,6002497,0.725,0.583,0.0427,0.300000,0.0665,0.762,79.984,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
5974,6002498,0.399,0.787,0.0499,0.019700,0.0685,0.572,117.089,0.000000,0.015385,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5975,6002499,0.376,0.435,0.0264,0.513000,0.1380,0.304,80.002,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [4]:
coverList = loadCoverGroups()

In [36]:
ids

0            23
1           105
2           118
3           120
4           292
         ...   
5972    6002496
5973    6002497
5974    6002498
5975    6002499
5976    6002500
Name: id, Length: 5977, dtype: int64

In [37]:
## Removing unamed and other unnecessary columns
df.reset_index(drop=True, inplace=True)
df = df.sort_values('id')
df.id = df.id.astype('int64')
# Saving idChords
ids = df.id
df = df.set_index('id')
df.reset_index(drop=True, inplace=True)
df

,danceability,energy,speechiness,acousticness,liveness,valence,tempo,0,1,2,...,mood_5,mood_6,mood_7,mood_8,mood_9,mood_10,mood_11,instrumentalness_cat_0,instrumentalness_cat_1,instrumentalness_cat_2
0,0.494,0.756,0.0432,0.001400,0.1640,0.589,133.337,0.000000,0.000000,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.582,0.744,0.0336,0.000458,0.0405,0.674,133.344,0.185185,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,0.614,0.755,0.0276,0.016200,0.0831,0.645,92.622,0.000000,0.000000,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.319,0.807,0.0388,0.001040,0.1310,0.601,118.097,0.029412,0.000000,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.496,0.527,0.0339,0.195000,0.1380,0.283,155.932,0.013889,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5972,0.509,0.537,0.0384,0.347000,0.0805,0.311,170.030,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5973,0.725,0.583,0.0427,0.300000,0.0665,0.762,79.984,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
5974,0.399,0.787,0.0499,0.019700,0.0685,0.572,117.089,0.000000,0.015385,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5975,0.376,0.435,0.0264,0.513000,0.1380,0.304,80.002,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [38]:
pd.DataFrame(coverList)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,180849,6001027,1686718.0,5000013.0,5000001.0,5000007.0,5000008.0,5000023.0,5000017.0,5000019.0,5000020.0,5000018.0,5000011.0,5000012.0
1,6001971,6001984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6002492,6002491,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6000699,6001165,6000182.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6000104,6000273,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6000768,6000784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6001931,6001748,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,6002043,6002073,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,6002244,6000194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,6001209,6000606,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
currentList = coverList[0]
currentList

[180849,
 6001027,
 1686718,
 5000013,
 5000001,
 5000007,
 5000008,
 5000023,
 5000017,
 5000019,
 5000020,
 5000018,
 5000011,
 5000012]

In [40]:
import gower

def applyGower(df,ids,id,n):
    indexToAnalyze = [i for i,x in enumerate(ids) if x == id][0]
    elementToAnalyze = df.loc[indexToAnalyze:indexToAnalyze,:]
    ranksDFGower = pd.DataFrame(gower.gower_topn(elementToAnalyze, df, n = n))
    songID = pd.DataFrame(ids[ranksDFGower['index']]).reset_index()
    ranksDFGower = ranksDFGower.merge(songID)
    ranksDFGower = ranksDFGower[['id','values']]
    return ranksDFGower

In [42]:
ranksDFGower = applyGower(df,ids, currentList[1],25)
ranksDFGower

,id,values
0,6001027,0.000000
1,180849,0.001754
2,6001258,0.002656
3,466649,0.002827
4,1029726,0.002885
5,5000017,0.002904
6,1473649,0.003287
7,2065851,0.003380
8,450492,0.003597
9,1713294,0.003601


In [43]:
ranksDFGower.loc[ranksDFGower.id.isin(currentList)]

,id,values
0,6001027,0.000000
1,180849,0.001754
5,5000017,0.002904
10,5000019,0.003660
15,5000001,0.003924
